In [18]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import os
import re
import xlwt

input1 = input('1. 유튜브에서 검색할 주제 키워드를 입력하세요(예:롯데마트) ')
#input1 = '카리나'
input2 = int(input('2. 위 주제로 댓글을 크롤링할 유튜브 영상은 몇건입니까?: '))

input3 = int(input('3. 각 동영상에서 추출할 댓글은 몇건입니까?: '))

f_name = input('5. 크롤링 결과를 저장할 폴더명만 쓰세요:')
f_dir = 'C:\\py_temp\\test\\'

n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+s+'-'+input1)
os.chdir(f_dir+s+'-'+input1)

ff_name=f_dir+s+'-'+input1+'\\'+s+'-'+input1+'.txt'
fc_name=f_dir+s+'-'+input1+'\\'+s+'-'+input1+'.csv'
fx_name=f_dir+s+'-'+input1+'\\'+s+'-'+input1+'.xls'

s_time = time.time( )

if os.path.isdir(f_dir):
    print('입력하신 경로가 존재하여 {} 폴더에 저장하겠습니다'.format(f_dir))
else:
    os.makedirs(f_dir)
    print('입력하신 경로가 존재하지 않아 {} 폴더를 생성했습니다'.format(f_dir))

s_time = time.time()

options = webdriver.ChromeOptions()
# 웹 드라이버 실행
driver_path = r"C:\py_temp\chromedriver\chromedriver.exe"
driver = webdriver.Chrome(driver_path)

# def auto_scroll(driver):
#     driver.execute_script("window.scrollTo(0, 3000)")
#     time.sleep(5)  # 5초 대기

# 유튜브 접속
driver.get("https://www.youtube.com")
time.sleep(2)
#창을 무조건 최대화 해야됩니다ㅎㅎ
driver.maximize_window()
commentCnt =0
cmtCnt = []
num = []
url = []
nickname = []
commentDate = []
commentContent = []
tmp_commentDate = ""
cont = ""

# 검색어 입력
element = driver.find_element(By.CSS_SELECTOR, "input#search")
element.send_keys(input1)
element.send_keys(Keys.ENTER)

full_html = driver.page_source
soup = BeautifulSoup(full_html, 'html.parser')

href_list = []


time.sleep(2)

judgment = 0

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# Find all 'a' elements with id="thumbnail"
a_elements = soup.find_all('a', class_="yt-simple-endpoint style-scope ytd-video-renderer")
href_list = [a['href'] for a in a_elements]
href_list = [href for href in href_list if '/shorts' not in href]

videoCnt = 0

# 비디오 검색 결과 수집
for link in href_list:
    print("videoCnt :", videoCnt)
    if videoCnt == input2:
        break;
    print("\n\nvideoCnt:",videoCnt,"\n\input2:",input2,"\n")
    time.sleep(2)

    new_link = "https://www.youtube.com" + link

    driver.get(new_link)
    
    time.sleep(1)

    driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
    #driver.execute_script("window.scrollTo(0, 100)")
    time.sleep(1)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    video_list = soup.find_all('ytd-comment-thread-renderer', 'style-scope ytd-item-section-renderer') #댓글
  
    if len(video_list) < input3:
        print("댓글 수가 입력 수 보다 적어 댓글 수 만큼 댓들이 크롤링 됩니다.")
        input3 = len(video_list)      
                
    print('주소:', new_link)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    video_list = soup.find_all('ytd-comment-thread-renderer', 'style-scope ytd-item-section-renderer')
    
    videoCnt += 1
    for j in video_list:  
        f = open(ff_name, 'a',encoding='UTF-8')
        if len(video_list) == 0:
            commentCnt +=1
            print("댓글이 없습니다.")
            continue;
#         if driver.current_url != new_link: 
#             videoCnt += 1 
        num.append(videoCnt)
        print('영상 수:', videoCnt)  
        f.write(str(videoCnt)+ '번째 영상의 ' +str(commentCnt+1) + '번째 댓글 추출\n')
        
        tmp_url = new_link
        url.append(tmp_url)
        print('url:', tmp_url) 
        f.write('1. URL 주소:' + url[commentCnt].strip() + '\n')
                       
        tmp_nickname = j.find('h3', class_='style-scope ytd-comment-renderer').get_text()
        nickname.append(tmp_nickname.strip())
        print('댓글 작성자명:', tmp_nickname)
        f.write('2. 댓글 작성자명:' + nickname[commentCnt].strip() + '\n')

        tmp_commentDate = j.find('a', 'yt-simple-endpoint style-scope yt-formatted-string').get_text()
        commentDate.append(tmp_commentDate.strip())
        print('작성일자:', tmp_commentDate)
        f.write('3. 댓글 작성일자:' + commentDate[commentCnt].strip() + '\n')
        
        
        comment = j.find('yt-formatted-string', class_='style-scope ytd-comment-renderer').get_text()
        commentContent.append(comment.strip())
        print('댓글 내용:', comment)
        print("commentCnt :", commentCnt)
        f.write('4. 댓글 내용:' + commentContent[commentCnt].strip() + '\n')
        f.write('\n')
        
        commentCnt +=1 
        cmtCnt.append(commentCnt)
            
        if commentCnt == input3:
            commentCnt = 0;
            break  
            
f.close()

result = pd.DataFrame()
result['번호'] = num
result['URL 주소'] = url
result['댓글작성자명'] = nickname
result['댓글 작성일자'] = commentDate
result['댓글 내용'] = commentContent


# 엑셀 파일 저장
result.to_excel(fx_name , index=False)

# csv 형태로 저장하기
result.to_csv(fc_name,encoding="utf-8-sig",index=False)

driver.close()


1. 유튜브에서 검색할 주제 키워드를 입력하세요(예:롯데마트) 야구
2. 위 주제로 댓글을 크롤링할 유튜브 영상은 몇건입니까?: 3
3. 각 동영상에서 추출할 댓글은 몇건입니까?: 3
5. 크롤링 결과를 저장할 폴더명만 쓰세요:C:\\py_temp\\test\\
입력하신 경로가 존재하여 C:\py_temp\test\ 폴더에 저장하겠습니다
videoCnt : 0


videoCnt: 0 
\input2: 3 

주소: https://www.youtube.com/watch?v=xuS_8gKWpuY&pp=ygUG7JW86rWs
영상 수: 1
url: https://www.youtube.com/watch?v=xuS_8gKWpuY&pp=ygUG7JW86rWs
댓글 작성자명: 


              구라핑
            


작성일자: 1일 전
댓글 내용: 유연성, 탄력은 진짜 대단하다...
commentCnt : 0
영상 수: 1
url: https://www.youtube.com/watch?v=xuS_8gKWpuY&pp=ygUG7JW86rWs
댓글 작성자명: 


              DarkRonier
            


작성일자: 1일 전(수정됨)
댓글 내용: 약물문제도 있었지만 타티스의 스타성을 사랑할수밖에 없다....
commentCnt : 1
영상 수: 1
url: https://www.youtube.com/watch?v=xuS_8gKWpuY&pp=ygUG7JW86rWs
댓글 작성자명: 


              버터_바닐라
            


작성일자: 1일 전
댓글 내용: 약물은 잘못한거지만

브레이크 세레모니, 게임중 나오는 흥, 덕아웃 응원단장
다른선수한테 볼수없는 운동 능력
수천억 슈퍼스타임에도 승부에 집중하는 모습
가장 중요한 결과물이 나온다는거

매력이 넘치는 선수임이 분명하다

+ 상의 단추 3개 풀은것도 용서됨
commentCnt : 2
videoCnt : 1


videoCnt: 1 
\input2: 3 